In [28]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

### Load training data

Load `train.csv` from Kaggle into a pandas DataFrame.

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Set up X and y

NOTE: Keras requires a `numpy` matrix, it doesn't work with `pandas`.

In [4]:
y= df['label']
X= df.drop(['label'], axis=1).values

In [5]:
type(X)

numpy.ndarray

### Preprocessing

1. When dealing with image data, you need to normalize your `X` by dividing each value by the max number of pixels (255).
2. Since this is a multiclass classification problem, keras needs `y` to be a one-hot encoded matrix

In [6]:
y = to_categorical(y)

In [7]:
X = X/255.

In [8]:
X

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

### Train/Test Split

We want to create a validation set that the model will never see to approximate how it's going to do with Kaggle's `test.csv`. Use `sklearn`'s `train_test_split` to do this.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [10]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(31500, 784) (10500, 784) (31500, 10) (10500, 10)


### Create your neural network

Create a neural network using the `Dense` and `Dropout` layers from `keras`. Your activation function for the final output layer needs to be `softmax` to accomidate the ten different classes.

In [11]:
model = Sequential()
model.add(Dense(X_train.shape[1], input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(.5))
model.add(Dense(100, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))



### Compile your model

Since this is a multiclass classification problem, your loss function is `categorical_crossentropy`.

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Fit the model

Use your X_test, y_test from the `train_test_split` step for the `validation_data` parameter.

In [13]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Train on 31500 samples, validate on 10500 samples
Epoch 1/10
31500/31500 [==============================] - 6s 197us/step - loss: 0.3210 - acc: 0.9023 - val_loss: 0.1459 - val_acc: 0.9565
Epoch 2/10
31500/31500 [==============================] - 9s 301us/step - loss: 0.1615 - acc: 0.9507 - val_loss: 0.1138 - val_acc: 0.9642
Epoch 3/10
31500/31500 [==============================] - 11s 342us/step - loss: 0.1331 - acc: 0.9580 - val_loss: 0.0875 - val_acc: 0.9726
Epoch 4/10
31500/31500 [==============================] - 11s 358us/step - loss: 0.1065 - acc: 0.9666 - val_loss: 0.0939 - val_acc: 0.9711
Epoch 5/10
31500/31500 [==============================] - 12s 366us/step - loss: 0.0923 - acc: 0.9697 - val_loss: 0.0879 - val_acc: 0.9707
Epoch 6/10
31500/31500 [==============================] - 12s 379us/step - loss: 0.0839 - acc: 0.9736 - val_loss: 0.0805 - val_acc: 0.9757
Epoch 7/10
31500/31500 [==============================] - 12s 389us/step - loss: 0.0787 - acc: 0.9730 - val_loss: 0.08

### Load in Kaggle's `test.csv`

Be sure to do the **same** preprocessing you did for your training `X`.

In [14]:
test = pd.read_csv('test.csv')
test = test.values/255.

### Create your predictions

Use `predict_classes` to get the actual numerical values (0-9).

In [15]:
pred = model.predict_classes(test)

28000/28000 [==============================] - 1s 43us/step


In [16]:
pred

array([2, 0, 9, ..., 3, 9, 2])

### Prepare your submission

1. Add your predictions to a column called `Label`
2. You'll need to manually create the `ImageId` column, which is just a list of 1..[NUMBER OF TEST SAMPLES]

In [17]:
submission = pd.DataFrame()


submission['ImageId'] = range(1,test.shape[0] + 1)

submission['Label']= pred

In [18]:
submission.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


### Create your submission csv

Remember to set `index=False`!

In [19]:
submission.to_csv('submission.csv', index=False)

In [20]:
# First score is 97.5%

# Third submission

In [21]:
model = Sequential()
model.add(Dense(X_train.shape[1], input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=100)

Train on 31500 samples, validate on 10500 samples
Epoch 1/50
31500/31500 [==============================] - 3s 106us/step - loss: 0.2656 - acc: 0.9205 - val_loss: 0.1187 - val_acc: 0.9646
Epoch 2/50
31500/31500 [==============================] - 4s 112us/step - loss: 0.1009 - acc: 0.9690 - val_loss: 0.1068 - val_acc: 0.9649
Epoch 3/50
31500/31500 [==============================] - 4s 133us/step - loss: 0.0618 - acc: 0.9810 - val_loss: 0.1208 - val_acc: 0.9645
Epoch 4/50
31500/31500 [==============================] - 4s 135us/step - loss: 0.0445 - acc: 0.9852 - val_loss: 0.0899 - val_acc: 0.9733
Epoch 5/50
31500/31500 [==============================] - 5s 145us/step - loss: 0.0342 - acc: 0.9886 - val_loss: 0.0944 - val_acc: 0.9722
Epoch 6/50
31500/31500 [==============================] - 5s 148us/step - loss: 0.0307 - acc: 0.9903 - val_loss: 0.1173 - val_acc: 0.9674
Epoch 7/50
31500/31500 [==============================] - 5s 157us/step - loss: 0.0255 - acc: 0.9913 - val_loss: 0.0923 - 

In [22]:
test = pd.read_csv('test.csv')
test = test.values/255.

pred = model.predict_classes(test)

28000/28000 [==============================] - 1s 35us/step


In [23]:
submission = pd.DataFrame()


submission['ImageId'] = range(1,test.shape[0] + 1)

submission['Label']= pred

In [24]:
submission.to_csv('submission1128.csv', index=False)

In [ ]:
# It is 98 %

# MNIST 1129

## First submission

In [25]:
df = pd.read_csv("train.csv")

In [26]:
y = to_categorical(df['label'].values) # to have list

X_train, X_test, y_train, y_test = train_test_split(df[df.columns[1:]].values, y)

X_train = X_train/255.
X_test = X_test/255.

X_train = X_train.reshape(X_train.shape[0], 28,28,1)
X_test = X_test.reshape(X_test.shape[0], 28,28,1)

In [29]:
model = Sequential()
model.add(Conv2D(15, kernel_size=(5,5), input_shape=(28,28,1), activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(30, kernel_size=(4,4), activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(45, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=40)

Train on 31500 samples, validate on 10500 samples
Epoch 1/40
31500/31500 [==============================] - 16s 511us/step - loss: 0.3378 - acc: 0.8937 - val_loss: 0.1185 - val_acc: 0.9632
Epoch 2/40
31500/31500 [==============================] - 17s 529us/step - loss: 0.0934 - acc: 0.9717 - val_loss: 0.0966 - val_acc: 0.9700
Epoch 3/40
31500/31500 [==============================] - 16s 514us/step - loss: 0.0627 - acc: 0.9805 - val_loss: 0.0665 - val_acc: 0.9813
Epoch 4/40
31500/31500 [==============================] - 16s 509us/step - loss: 0.0512 - acc: 0.9843 - val_loss: 0.0506 - val_acc: 0.9839
Epoch 5/40
31500/31500 [==============================] - 17s 528us/step - loss: 0.0433 - acc: 0.9866 - val_loss: 0.0563 - val_acc: 0.9820
Epoch 6/40
31500/31500 [==============================] - 17s 536us/step - loss: 0.0343 - acc: 0.9887 - val_loss: 0.0469 - val_acc: 0.9871
Epoch 7/40
31500/31500 [==============================] - 16s 504us/step - loss: 0.0278 - acc: 0.9911 - val_loss: 0.

In [30]:
test = pd.read_csv('test.csv')
test = test.values/255.
test = test.reshape(test.shape[0], 28,28,1)

pred = model.predict_classes(test)

28000/28000 [==============================] - 4s 151us/step


In [31]:
submission = pd.DataFrame()


submission['ImageId'] = range(1,test.shape[0] + 1)

submission['Label']= pred

submission.to_csv('submission1.csv', index=False)

In [32]:
# It's 0.98371

# Second submission

In [34]:
model = Sequential()
model.add(Conv2D(15, kernel_size=(5,5), input_shape=(28,28,1), activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(30, kernel_size=(4,4), activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(45, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=40)

Train on 31500 samples, validate on 10500 samples
Epoch 1/40
31500/31500 [==============================] - 18s 580us/step - loss: 0.2776 - acc: 0.9136 - val_loss: 0.0979 - val_acc: 0.9690
Epoch 2/40
31500/31500 [==============================] - 18s 581us/step - loss: 0.0853 - acc: 0.9743 - val_loss: 0.0759 - val_acc: 0.9763
Epoch 3/40
31500/31500 [==============================] - 18s 583us/step - loss: 0.0598 - acc: 0.9815 - val_loss: 0.0499 - val_acc: 0.9845
Epoch 4/40
31500/31500 [==============================] - 19s 589us/step - loss: 0.0474 - acc: 0.9850 - val_loss: 0.0669 - val_acc: 0.9803
Epoch 5/40
31500/31500 [==============================] - 18s 581us/step - loss: 0.0424 - acc: 0.9866 - val_loss: 0.0689 - val_acc: 0.9795
Epoch 6/40
31500/31500 [==============================] - 19s 590us/step - loss: 0.0338 - acc: 0.9894 - val_loss: 0.0515 - val_acc: 0.9860
Epoch 7/40
31500/31500 [==============================] - 19s 594us/step - loss: 0.0295 - acc: 0.9904 - val_loss: 0.

In [35]:
test = pd.read_csv('test.csv')
test = test.values/255.
test = test.reshape(test.shape[0], 28,28,1)

pred = model.predict_classes(test)

28000/28000 [==============================] - 5s 161us/step


In [36]:
submission = pd.DataFrame()


submission['ImageId'] = range(1,test.shape[0] + 1)

submission['Label']= pred

submission.to_csv('submission6.csv', index=False)

In [37]:
# This time is 0.98457